In [40]:
search_query='cat:cs.CV+OR+cat:cs.LG+OR+cat:cs.CL+OR+cat:cs.AI+OR+cat:cs.NE+OR+cat:cs.RO'
start_index=0
base_url = 'http://export.arxiv.org/api/query?'
add_url = 'search_query=%s&sortBy=lastUpdatedDate&start=%d&max_results=2' % (search_query, start_index)
#add_url = 'search_query=%s&sortBy=submittedDate&start=%d&max_results=100' % (search_query, start_index)
search_query = base_url + add_url

In [41]:
search_query

'http://export.arxiv.org/api/query?search_query=cat:cs.CV+OR+cat:cs.LG+OR+cat:cs.CL+OR+cat:cs.AI+OR+cat:cs.NE+OR+cat:cs.RO&sortBy=lastUpdatedDate&start=0&max_results=2'

In [42]:
import time
import logging
import urllib.request
import feedparser
from collections import OrderedDict

In [43]:
with urllib.request.urlopen(search_query) as url:
    response = url.read()

In [44]:
response

b'<?xml version="1.0" encoding="UTF-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <link href="http://arxiv.org/api/query?search_query%3Dcat%3Acs.CV%20OR%20cat%3Acs.LG%20OR%20cat%3Acs.CL%20OR%20cat%3Acs.AI%20OR%20cat%3Acs.NE%20OR%20cat%3Acs.RO%26id_list%3D%26start%3D0%26max_results%3D2" rel="self" type="application/atom+xml"/>\n  <title type="html">ArXiv Query: search_query=cat:cs.CV OR cat:cs.LG OR cat:cs.CL OR cat:cs.AI OR cat:cs.NE OR cat:cs.RO&amp;id_list=&amp;start=0&amp;max_results=2</title>\n  <id>http://arxiv.org/api/JAnYJ7RkReDnT8m1ZaByqaCEdn0</id>\n  <updated>2022-06-28T00:00:00-04:00</updated>\n  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">230672</opensearch:totalResults>\n  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>\n  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">2</opensearch:itemsPerPage>\n  <entry>\n    <id>http://arxiv.org/abs/2206.

In [45]:
def parse_response(response):

    out = []
    parse = feedparser.parse(response)
    for e in parse.entries:
        j = encode_feedparser_dict(e)
        # extract / parse id information
        idv, rawid, version = parse_arxiv_url(j['id'])
        j['_idv']= idv
        j['_id'] = rawid
        j['_version'] = version
        j['_time'] = time.mktime(j['updated_parsed'])
        j['_time_str'] = time.strftime('%b %d %Y', j['updated_parsed'])
        # delete apparently spurious and redundant information
        del j['summary_detail']
        del j['title_detail']
        out.append(j)

    return out
def parse_arxiv_url(url):
    """
    examples is http://arxiv.org/abs/1512.08756v2
    we want to extract the raw id (1512.08756) and the version (2)
    """
    ix = url.rfind('/')
    assert ix >= 0, 'bad url: ' + url
    idv = url[ix+1:] # extract just the id (and the version)
    parts = idv.split('v')
    assert len(parts) == 2, 'error splitting id and version in idv string: ' + idv
    return idv, parts[0], int(parts[1])

def encode_feedparser_dict(d):
    """ helper function to strip feedparser objects using a deep copy """
    if isinstance(d, feedparser.FeedParserDict) or isinstance(d, dict):
        return {k: encode_feedparser_dict(d[k]) for k in d.keys()}
    elif isinstance(d, list):
        return [encode_feedparser_dict(k) for k in d]
    else:
        return d

In [46]:
papers = parse_response(response)

In [47]:
papers

[{'id': 'http://arxiv.org/abs/2206.13502v1',
  'guidislink': True,
  'link': 'http://arxiv.org/abs/2206.13502v1',
  'updated': '2022-06-27T17:59:57Z',
  'updated_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
  'published': '2022-06-27T17:59:57Z',
  'published_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
  'title': 'Programmatic Concept Learning for Human Motion Description and Synthesis',
  'summary': 'We introduce Programmatic Motion Concepts, a hierarchical motion\nrepresentation for human actions that captures both low-level motion and\nhigh-level description as motion concepts. This representation enables human\nmotion description, interactive editing, and controlled synthesis of novel\nvideo sequences within a single framework. We present an architecture that\nlearns this concept representation from paired vi

In [48]:
papers[0]

{'id': 'http://arxiv.org/abs/2206.13502v1',
 'guidislink': True,
 'link': 'http://arxiv.org/abs/2206.13502v1',
 'updated': '2022-06-27T17:59:57Z',
 'updated_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
 'published': '2022-06-27T17:59:57Z',
 'published_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
 'title': 'Programmatic Concept Learning for Human Motion Description and Synthesis',
 'summary': 'We introduce Programmatic Motion Concepts, a hierarchical motion\nrepresentation for human actions that captures both low-level motion and\nhigh-level description as motion concepts. This representation enables human\nmotion description, interactive editing, and controlled synthesis of novel\nvideo sequences within a single framework. We present an architecture that\nlearns this concept representation from paired video and a

In [106]:
a=response2.json()['articles'][11]

In [107]:
a

{'source': {'id': None, 'name': 'Harvard Business Review'},
 'author': None,
 'title': 'Scaling a Fintech Startup for the Greater Good',
 'description': 'Can fintech startup Esusu use innovation to promote financial inclusion in America?',
 'url': 'https://hbr.org/podcast/2022/06/scaling-a-fintech-startup-for-the-greater-good',
 'urlToImage': 'https://hbr.org/resources/images/article_assets/2019/03/Cold_Call_WP_Wide_1200.png',
 'publishedAt': '2022-06-28T13:00:00Z',
 'content': 'BRIAN KENNY: March 24, 2023 is going to be a big day in Sweden because that is the date that Sweden will become the first cashless society in the world. On that day, you will need either a credit car… [+21949 chars]'}

In [108]:
del a['urlToImage']
del a['urlToImage']

In [163]:
pd.to_datetime(a['publishedAt']).value

1656421200000000000

In [149]:
a['publishedAt']

'2022-06-28T13:00:00Z'

In [155]:
b=pd.to_datetime(a['publishedAt'])

In [168]:
b.ctime()

'Tue Jun 28 13:00:00 2022'

{'id': 'http://arxiv.org/abs/2206.13502v1',
 'guidislink': True,
 'link': 'http://arxiv.org/abs/2206.13502v1',
 'updated': '2022-06-27T17:59:57Z',
 'updated_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
 'published': '2022-06-27T17:59:57Z',
 'published_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
 'title': 'Programmatic Concept Learning for Human Motion Description and Synthesis',
 'summary': 'We introduce Programmatic Motion Concepts, a hierarchical motion\nrepresentation for human actions that captures both low-level motion and\nhigh-level description as motion concepts. This representation enables human\nmotion description, interactive editing, and controlled synthesis of novel\nvideo sequences within a single framework. We present an architecture that\nlearns this concept representation from paired video and a

In [110]:
def t():
    for i in range(4):
        author_str=' '.join([a['name'] for a in papers[i]['authors']])

        yield ' '.join([papers[i]['title'], papers[i]['summary'], author_str])

In [111]:
for i in t():
    print(i)
    break

Programmatic Concept Learning for Human Motion Description and Synthesis We introduce Programmatic Motion Concepts, a hierarchical motion
representation for human actions that captures both low-level motion and
high-level description as motion concepts. This representation enables human
motion description, interactive editing, and controlled synthesis of novel
video sequences within a single framework. We present an architecture that
learns this concept representation from paired video and action sequences in a
semi-supervised manner. The compactness of our representation also allows us to
present a low-resource training recipe for data-efficient learning. By
outperforming established baselines, especially in the small data regime, we
demonstrate the efficiency and effectiveness of our framework for multiple
applications. Sumith Kulal Jiayuan Mao Alex Aiken Jiajun Wu


In [116]:
' '.join([a['name'] for a in papers[1]['authors']])

'Ryan Burgert Jinghuan Shang Xiang Li Michael Ryoo'

In [131]:
a[0]

{'title': '‘Deadpool’ Writers Share Their Suggestions Of Who Should Play Wolverine In The MCU',
 'description': 'Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool. During an interview with BroBible’s Post-Credit Podcast following the release of Spiderhead, we asked the pair if they had any suggestions of who should play…',
 'url': 'https://brobible.com/culture/article/deadpool-writers-wolverine-casting-choices-mcu/',
 'publishedAt': '2022-06-28T17:36:05Z',
 '_id': 'https://brobible.com/culture/article/deadpool-writers-wolverine-casting-choices-mcu/',
 '_time_str': 'Tue Jun 28 17:36:05 2022',
 '_time': 1656437765000000000,
 'content': 'Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool. During an interview with BroBible’s Post-Credit Podcast following the release of Spiderhead, we asked the pair if they had any suggestions of who should play…',
 'authors': 'Eric Italiano'}

In [119]:
response2.json()['articles'][0]

{'source': {'id': None, 'name': 'New York Post'},
 'author': 'Associated Press',
 'title': 'Stocks erase early gains as inflation concerns persist on Wall Street',
 'description': 'The S&P 500 fell 0.8% as of noon Eastern. The Dow Jones Industrial Average fell 150 points, or 0.5%, to 31,644 and the Nasdaq fell 1.6%.',
 'url': 'https://nypost.com/2022/06/28/stocks-fall-on-wall-street-as-inflation-concerns-persist/',
 'urlToImage': 'https://nypost.com/wp-content/uploads/sites/2/2022/06/dow-markets-fall.jpg?quality=75&strip=all&w=1024',
 'publishedAt': '2022-06-28T17:08:00Z',
 'content': 'Stocks fell in midday trading on Wall Street Tuesday as the broader market remains gripped by uncertainty over pervasive inflation, rising interest rates and the potential for a recession.\r\nThe S&amp… [+2992 chars]'}

In [129]:
def t2():
    for i in range(4):
        author_str=a[i]['authors']

        yield ' '.join([a[i]['title'], a[i]['content'], author_str])

In [130]:
for i in t2():
    print(i)
    break

‘Deadpool’ Writers Share Their Suggestions Of Who Should Play Wolverine In The MCU Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool. During an interview with BroBible’s Post-Credit Podcast following the release of Spiderhead, we asked the pair if they had any suggestions of who should play… Eric Italiano


In [109]:
response2.json()['articles']

[{'source': {'id': None, 'name': 'New York Post'},
  'author': 'Associated Press',
  'title': 'Stocks erase early gains as inflation concerns persist on Wall Street',
  'description': 'The S&P 500 fell 0.8% as of noon Eastern. The Dow Jones Industrial Average fell 150 points, or 0.5%, to 31,644 and the Nasdaq fell 1.6%.',
  'url': 'https://nypost.com/2022/06/28/stocks-fall-on-wall-street-as-inflation-concerns-persist/',
  'urlToImage': 'https://nypost.com/wp-content/uploads/sites/2/2022/06/dow-markets-fall.jpg?quality=75&strip=all&w=1024',
  'publishedAt': '2022-06-28T17:08:00Z',
  'content': 'Stocks fell in midday trading on Wall Street Tuesday as the broader market remains gripped by uncertainty over pervasive inflation, rising interest rates and the potential for a recession.\r\nThe S&amp… [+2992 chars]'},
 {'source': {'id': None, 'name': 'MacRumors'},
  'author': 'Juli Clover',
  'title': 'Apple Seeds Fourth Betas of iOS 15.6 and iPadOS 15.6 to Developers',
  'description': 'Apple 

In [126]:
pd.to_datetime(startdate).strftime('%Y-%m-%d')

'2022-01-06'

In [115]:
from datetime import timedelta, date
date_1 = date.today()

end_date = date_1 + datetime.timedelta(days=3)

In [116]:
date_1

datetime.date(2022, 6, 28)

In [182]:
import pandas as pd
startdate = "01/06/2022"
enddate = pd.to_datetime(startdate) + pd.DateOffset(days=2)
import requests
url = ('https://newsapi.org/v2/everything?'
       'q=Apple&'
       'from={startdate}&'
       'to={end_date}&'
       'sortBy=publishedAt&'
       'apiKey=ee48498e46f04eb38a222ad5170fbe8b')
response2 = requests.get(url)
print(response2.json())

{'status': 'ok', 'totalResults': 60405, 'articles': [{'source': {'id': None, 'name': 'Caschys Blog'}, 'author': 'Felix Frank', 'title': 'Samsung Galaxy XCover6 Pro: Das sind die technischen Spezifikationen', 'description': 'Zu Samsungs geplantem Rugged-Smartphone, dem Galaxy XCover6 Pro konnten wir euch bereits erste Render-Bilder zeigen. Auch Samsung macht keinen Hehl um die Optik des Smartphones und hat jene bereits im Zuge der Event-Ankündigung für den 13. Juli 2022 gezeigt. …', 'url': 'https://stadt-bremerhaven.de/samsung-galaxy-xcover6-pro-das-sind-die-technischen-spezifikationen/', 'urlToImage': 'https://stadt-bremerhaven.de/wp-content/uploads/2022/06/FWV_4pIWYAEuvvc.jpg', 'publishedAt': '2022-06-28T16:54:15Z', 'content': 'Zu Samsungs geplantem Rugged-Smartphone, dem Galaxy XCover6 Pro konnten wir euch bereits erste Render-Bilder zeigen. Auch Samsung macht keinen Hehl um die Optik des Smartphones und hat jene bereits i… [+2492 chars]'}, {'source': {'id': 'der-tagesspiegel', 'name

In [130]:
response2.json()['articles'][10]

{'source': {'id': None, 'name': 'Cult of Mac'},
 'author': 'Ed Hardy',
 'title': 'Apple TV+ subscribers think it’s a better value than Netflix',
 'description': 'Apple TV+ subscribers are much more satisfied with the streaming service than they were a year ago, according to a survey of viewers.\xa0The home of Ted Lasso and Severed even made gains on\xa0Netflix – streaming’s 500-point gorilla – in a couple of categories, inc…',
 'url': 'https://www.cultofmac.com/782313/apple-tv-subscribers-think-its-a-better-value-than-netflix/',
 'urlToImage': 'https://www.cultofmac.com/wp-content/uploads/2022/06/EA45CC2B-75EE-460B-80D3-FA40D30CFE30.jpeg',
 'publishedAt': '2022-06-28T16:00:28Z',
 'content': 'Apple TV+ subscribers are much more satisfied with the streaming service than they were a year ago, according to a survey of viewers.\xa0The home of Ted Lasso and Severed even made gains on\xa0Netflix stre… [+1853 chars]'}

In [84]:
response.json()['articles'][0]

{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'Mat Smith',
 'title': 'The Morning After: Valve warns you, again, against modifying your Steam Deck',
 'description': 'Valve loves to warn people about\r\n the risks of do-it-yourself Steam Deck maintenance, and this time it’s reminding you that trying to upgrade the built-in storage could lead to problems.Following a PC Gamer article on modding the Steam Deck, Valve hardware d…',
 'url': 'https://www.engadget.com/the-morning-after-valve-warns-you-again-against-modifying-your-steam-deck-111540388.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2022-02/f0a6bcb2-95ae-11ec-bfdd-2b873683c54e',
 'publishedAt': '2022-06-28T11:15:40Z',
 'content': 'Valve loves to warn people about\r\n the risks of do-it-yourself Steam Deck maintenance, and this time its reminding you that trying to upgrade the built-in storage could lead to problems.\r\nFollowing a… [+4061 chars]'}

In [30]:
aa=response2.json()['articles']

In [33]:
' '.join([a for a in aa['authors']])

TypeError: list indices must be integers or slices, not str

In [36]:
aa['summary']

TypeError: list indices must be integers or slices, not str

In [38]:
aa['description']

TypeError: list indices must be integers or slices, not str

In [49]:
papers

[{'id': 'http://arxiv.org/abs/2206.13502v1',
  'guidislink': True,
  'link': 'http://arxiv.org/abs/2206.13502v1',
  'updated': '2022-06-27T17:59:57Z',
  'updated_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
  'published': '2022-06-27T17:59:57Z',
  'published_parsed': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=27, tm_hour=17, tm_min=59, tm_sec=57, tm_wday=0, tm_yday=178, tm_isdst=0),
  'title': 'Programmatic Concept Learning for Human Motion Description and Synthesis',
  'summary': 'We introduce Programmatic Motion Concepts, a hierarchical motion\nrepresentation for human actions that captures both low-level motion and\nhigh-level description as motion concepts. This representation enables human\nmotion description, interactive editing, and controlled synthesis of novel\nvideo sequences within a single framework. We present an architecture that\nlearns this concept representation from paired vi

In [58]:
' '.join([a['name'] for a in papers[0]['authors']])

'Sumith Kulal Jiayuan Mao Alex Aiken Jiajun Wu'

In [56]:
papers[0]['authors']

[{'name': 'Sumith Kulal'},
 {'name': 'Jiayuan Mao'},
 {'name': 'Alex Aiken'},
 {'name': 'Jiajun Wu'}]

In [64]:
aa[2]['author']

'José Adorno'

In [65]:
aa[0]

{'source': {'id': None, 'name': 'New York Post'},
 'author': 'Associated Press',
 'title': 'Stocks erase early gains as inflation concerns persist on Wall Street',
 'description': 'The S&P 500 fell 0.8% as of noon Eastern. The Dow Jones Industrial Average fell 150 points, or 0.5%, to 31,644 and the Nasdaq fell 1.6%.',
 'url': 'https://nypost.com/2022/06/28/stocks-fall-on-wall-street-as-inflation-concerns-persist/',
 'urlToImage': 'https://nypost.com/wp-content/uploads/sites/2/2022/06/dow-markets-fall.jpg?quality=75&strip=all&w=1024',
 'publishedAt': '2022-06-28T17:08:00Z',
 'content': 'Stocks fell in midday trading on Wall Street Tuesday as the broader market remains gripped by uncertainty over pervasive inflation, rising interest rates and the potential for a recession.\r\nThe S&amp… [+2992 chars]'}

In [66]:
aa

[{'source': {'id': None, 'name': 'New York Post'},
  'author': 'Associated Press',
  'title': 'Stocks erase early gains as inflation concerns persist on Wall Street',
  'description': 'The S&P 500 fell 0.8% as of noon Eastern. The Dow Jones Industrial Average fell 150 points, or 0.5%, to 31,644 and the Nasdaq fell 1.6%.',
  'url': 'https://nypost.com/2022/06/28/stocks-fall-on-wall-street-as-inflation-concerns-persist/',
  'urlToImage': 'https://nypost.com/wp-content/uploads/sites/2/2022/06/dow-markets-fall.jpg?quality=75&strip=all&w=1024',
  'publishedAt': '2022-06-28T17:08:00Z',
  'content': 'Stocks fell in midday trading on Wall Street Tuesday as the broader market remains gripped by uncertainty over pervasive inflation, rising interest rates and the potential for a recession.\r\nThe S&amp… [+2992 chars]'},
 {'source': {'id': None, 'name': 'MacRumors'},
  'author': 'Juli Clover',
  'title': 'Apple Seeds Fourth Betas of iOS 15.6 and iPadOS 15.6 to Developers',
  'description': 'Apple 

In [17]:
import requests
import os
from pprint import pprint

apikey = 'Csth7Lg8qvQzEB4MxFHiGJ7jd9z7HndP'

# Top Stories:
# https://developer.nytimes.com/docs/top-stories-product/1/overview
section = "science"
query_url = f"https://api.nytimes.com/svc/topstories/v2/{section}.json?api-key={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'fault': {'detail': {'errorcode': 'oauth.v2.InvalidApiKeyForGivenResource'},
           'faultstring': 'Invalid ApiKey for given resource'}}


In [22]:
apikey = 'Csth7Lg8qvQzEB4MxFHiGJ7jd9z7HndP'
query_url = f"https://api.nytimes.com/svc/news/v3/content/nyt/world.json?api-key={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'copyright': 'Copyright (c) 2022 The New York Times Company.  All Rights '
              'Reserved.',
 'num_results': 500,
 'results': [{'abstract': '',
              'byline': 'BY GENEVIEVE GLATSKY',
              'created_date': '2022-06-28T09:15:58-04:00',
              'des_facet': None,
              'first_published_date': '2022-06-28T09:15:58-04:00',
              'geo_facet': None,
              'item_type': 'Article',
              'kicker': '',
              'material_type_facet': 'Live Blog Post',
              'multimedia': [{'caption': 'Archbishop Desmond Tutu of South '
                                         'Africa during the 1995 Truth and '
                                         'Reconciliation Commission in South '
                                         'Africa.',
                              'copyright': 'Philip Littleton/Agence '
                                           'France-Presse — Getty Images',
                              'format': 'Standard Thumb

                                         'the Revolutionary Armed Forces of '
                                         'Colombia, leaving more than 100 '
                                         'people dead.',
                              'copyright': 'Ricardo Mazalan/Associated Press',
                              'format': 'mediumThreeByTwo210',
                              'height': 140,
                              'subtype': 'photo',
                              'type': 'image',
                              'url': 'https://static01.nyt.com/images/2022/06/28/world/28colombia-report-conflict-history/merlin_164638266_caedba34-4768-4137-9139-9ad0f18c7b5c-mediumThreeByTwo210.jpg',
                              'width': 210},
                             {'caption': 'A church in Bojayá, Colombia, that '
                                         'was destroyed in a 2002 attack by '
                                         'the Revolutionary Armed Forces of '
                       

                                         'Russia.',
                              'copyright': 'Reuters',
                              'format': 'mediumThreeByTwo210',
                              'height': 140,
                              'subtype': 'photo',
                              'type': 'image',
                              'url': 'https://static01.nyt.com/images/2022/06/28/us/28ukraine-briefing-oil-cap/28ukraine-briefing-oil-cap-mediumThreeByTwo210.jpg',
                              'width': 210},
                             {'caption': 'An oil refinery in Volgograd, '
                                         'Russia.',
                              'copyright': 'Reuters',
                              'format': 'mediumThreeByTwo440',
                              'height': 293,
                              'subtype': 'photo',
                              'type': 'image',
                              'url': 'https://static01.nyt.com/images/2022/06/28/us/28ukraine-b

                            'Grain',
                            'Food',
                            'United States Politics and Government',
                            'Food Insecurity'],
              'first_published_date': '2022-06-28T04:45:06-04:00',
              'geo_facet': ['Russia', 'Ukraine'],
              'item_type': 'Article',
              'kicker': '',
              'material_type_facet': 'News',
              'multimedia': [{'caption': 'President Biden with President '
                                         'Emmanuel Macron of France at the '
                                         'Group of 7 summit in Germany on '
                                         'Monday.',
                              'copyright': 'Kenny Holston for The New York '
                                           'Times',
                              'format': 'Standard Thumbnail',
                              'height': 75,
                              'subtype': 'photo',
                 

In [34]:
pip install newsapi-python

Note: you may need to restart the kernel to use updated packages.


In [35]:
from newsapi import NewsApiClient

In [46]:
newsapi = NewsApiClient(api_key='ee48498e46f04eb38a222ad5170fbe8b')

In [64]:
all_articles = newsapi.get_everything(q='bitcoin',
                                      sources='bbc-news',
                                      domains='bbc.co.uk',
                                      #from_param='2022-05-01',
                                      #to='2022-05-12',
                                      #category='general',
                                      language='en',
                                      sort_by='relevancy',
                                      #country='ua',
                                      page=2)

In [65]:
all_articles

{'status': 'ok', 'totalResults': 6, 'articles': []}

In [56]:
sources = newsapi.get_sources()

In [59]:
sources['sources']

[{'id': 'abc-news',
  'name': 'ABC News',
  'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.',
  'url': 'https://abcnews.go.com',
  'category': 'general',
  'language': 'en',
  'country': 'us'},
 {'id': 'abc-news-au',
  'name': 'ABC News (AU)',
  'description': "Australia's most trusted source of local, national and world news. Comprehensive, independent, in-depth analysis, the latest business, sport, weather and more.",
  'url': 'http://www.abc.net.au/news',
  'category': 'general',
  'language': 'en',
  'country': 'au'},
 {'id': 'aftenposten',
  'name': 'Aftenposten',
  'description': 'Norges ledende nettavis med alltid oppdaterte nyheter innenfor innenriks, utenriks, sport og kultur.',
  'url': 'https://www.aftenposten.no',
  'category': 'general',
  'language': 'no',
  'country': 'no'},
 {'id': 'al-jazeera-english',
  'name': 'Al Jazeera English',
  'description': 'News, analysis from the Middle East and wo

In [1]:
import pandas as pd
startdate = "01/06/2022"
enddate = pd.to_datetime(startdate) + pd.DateOffset(days=2)
import requests
url = ('https://newsapi.org/v2/everything?'
       'q=Apple&'
       'from={startdate}&'
       'to={end_date}&'
       'sortBy=publishedAt&'
       'apiKey=ee48498e46f04eb38a222ad5170fbe8b')
response2 = requests.get(url)

In [3]:
response2['articles']

TypeError: 'Response' object is not subscriptable

In [12]:
def get_response(search_query, start_index=0):
    import requests
    # """ pings arxiv.org API to fetch a batch of 100 papers """
    # # fetch raw response
    # base_url = 'http://export.arxiv.org/api/query?'
    # add_url = 'search_query=%s&sortBy=lastUpdatedDate&start=%d&max_results=100' % (search_query, start_index)
    # #add_url = 'search_query=%s&sortBy=submittedDate&start=%d&max_results=100' % (search_query, start_index)
    # search_query = base_url + add_url
    # logger.debug(f"Searching arxiv for {search_query}")
    # with urllib.request.urlopen(search_query) as url:
    #     response = url.read()

    # if url.status != 200:
    #     logger.error(f"arxiv did not return status 200 response")

    
    startdate = "01/06/2022"
    end_date = pd.to_datetime(startdate) + pd.DateOffset(days=start_index)
    
    url = ('https://newsapi.org/v2/everything?'
        'q=Apple&'
        'from={startdate}&'
        'to={end_date}&'
        'sortBy=publishedAt&'
        'apiKey=ee48498e46f04eb38a222ad5170fbe8b')
    response2 = requests.get(url).json()
    return response2['articles']

In [5]:
start_index=0
import requests

In [7]:
    startdate = "01/06/2022"
    end_date = pd.to_datetime(startdate) + pd.DateOffset(days=start_index)
    
    url = ('https://newsapi.org/v2/everything?'
        'q=Apple&'
        'from={startdate}&'
        'to={end_date}&'
        'sortBy=publishedAt&'
        'apiKey=ee48498e46f04eb38a222ad5170fbe8b')
    response2 = requests.get(url)

In [8]:
response2

<Response [200]>

In [67]:
def parse_response(response):

    out = []
    # parse = feedparser.parse(response)
    for e in response:
    #     j = encode_feedparser_dict(e)
    #     # extract / parse id information
    #     idv, rawid, version = parse_arxiv_url(j['id'])
    #     j['_idv']= idv
    #     j['_id'] = rawid
    #     j['_version'] = version
    #     j['_time'] = time.mktime(j['updated_parsed'])
    #     j['_time_str'] = time.strftime('%b %d %Y', j['updated_parsed'])
    #     # delete apparently spurious and redundant information
    #     del j['summary_detail']
    #     del j['title_detail']
    #     out.append(j)+
        del e['urlToImage']
        del e['source']
        
        del e['content']

        e['_id']=  e['url']
        e['_time_str']=  pd.to_datetime(e['publishedAt']).ctime()
        e['_time']= pd.to_datetime(e['publishedAt']).value
        e['content']=  e['description']
        e['authors']=  e['author']
        del e['author']

        out.append(e)

    return response

In [68]:
resp = get_response(search_query=None, start_index=0)

In [69]:
resp

[{'source': {'id': None, 'name': 'BroBible'},
  'author': 'Eric Italiano',
  'title': '‘Deadpool’ Writers Share Their Suggestions Of Who Should Play Wolverine In The MCU',
  'description': 'Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool. During an interview with BroBible’s Post-Credit Podcast following the release of Spiderhead, we asked the pair if they had any suggestions of who should play…',
  'url': 'https://brobible.com/culture/article/deadpool-writers-wolverine-casting-choices-mcu/',
  'urlToImage': 'https://brobible.com/wp-content/uploads/2022/06/deadpool-writers-wolverine.jpg',
  'publishedAt': '2022-06-28T17:36:05Z',
  'content': 'Getty Image/Epic Games/Marvel\r\n<ul><li>Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool.</li><li>During an interview with BroBible’s Post-Credit Po… [+2081 chars]'},
 {'source': {'id': None, 'name': 'eWeek'},
  'author': 'James Maguire',
  'ti

In [70]:
a=parse_response(resp)

In [77]:
a[0]

{'title': '‘Deadpool’ Writers Share Their Suggestions Of Who Should Play Wolverine In The MCU',
 'description': 'Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool. During an interview with BroBible’s Post-Credit Podcast following the release of Spiderhead, we asked the pair if they had any suggestions of who should play…',
 'url': 'https://brobible.com/culture/article/deadpool-writers-wolverine-casting-choices-mcu/',
 'publishedAt': '2022-06-28T17:36:05Z',
 '_id': 'https://brobible.com/culture/article/deadpool-writers-wolverine-casting-choices-mcu/',
 '_time_str': 'Tue Jun 28 17:36:05 2022',
 '_time': 1656437765000000000,
 'content': 'Rhett Reese and Paul Wernick are the screenwriting team behind films such as Spiderhead and Deadpool. During an interview with BroBible’s Post-Credit Podcast following the release of Spiderhead, we asked the pair if they had any suggestions of who should play…',
 'authors': 'Eric Italiano'}

In [75]:
keys=[a[i]['url'] for i in range(len(a))]

In [76]:
keys

['https://brobible.com/culture/article/deadpool-writers-wolverine-casting-choices-mcu/',
 'https://www.eweek.com/security/preventing-fraud-in-sme-market/',
 'https://devblogs.microsoft.com/dotnet/devops-for-dotnet-maui/',
 'https://mymodernmet.com/us-companies-travel-reimbursement-abortion-care/',
 'https://www.iphoneincanada.ca/news/apple-dual-pro-display-xdr-stand/',
 'https://www.redmondpie.com/vankyo-leisure-495w-is-the-first-dolby-audio-fhd-1080p-projector-features-mobile-mirroring-up-to-220-inch-screen-and-much-more/',
 'https://www.pajiba.com/celebrities_are_better_than_you/dakota-johnson-blames-the-incredibly-cheesy-el-james-for-those-disastrous-big-naked-movies.php',
 'https://www.stltoday.com/news/local/state-and-regional/state-urged-safety-upgrades-for-site-of-fatal-amtrak-wreck-death-toll-now-at-four/article_d87b3eaa-192b-5e9a-8bc7-ff9446220da2.html',
 'https://www.zdnet.com/article/tg-pro-must-have-app-for-your-new-m2-macs/',
 'https://www.macitynet.it/?p=1129433',
 'https

In [ ]:
        for p in keys:
            d = pdb[p]
            author_str =d['authors']#' '.join([a for a in d['authors']])
            yield ' '.join([d['title'], d['summary'], author_str])

In [ ]:
for i in keys:
    pdb[i]=

In [ ]:
' '.join([a['name'] for a in d['authors']])

In [85]:
response2.json()['articles'][10]

{'source': {'id': None, 'name': 'Deadline'},
 'author': 'peterdeadline',
 'title': 'NBC Digs Up Dino Docuseries ‘Surviving Earth’ From ‘Walking With Dinosaurs’ Creator',
 'description': 'The creator of the critically acclaimed BBC natural history series Walking with Dinosaurs has landed a new eight-part docuseries at NBC. The broadcast network has ordered Surviving Earth that explores the bizarre and amazing creatures and breathtaking landsca…',
 'url': 'https://deadline.com/2022/06/nbc-dinosaur-docuseries-surviving-earth-1235053396/',
 'urlToImage': 'https://deadline.com/wp-content/uploads/2022/06/Collage-Maker-28-Jun-2022-09.33-AM.jpg?w=1000',
 'publishedAt': '2022-06-28T17:00:18Z',
 'content': 'The creator of the critically acclaimed BBC natural history series Walking with Dinosaurs has landed a new eight-part docuseries at NBC.\r\nThe broadcast network has ordered Surviving Earth that explor… [+2252 chars]'}

In [133]:
response3 = requests.get('https://nypost.com/2022/06/28/stocks-fall-on-wall-street-as-inflation-concerns-persist').json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [136]:
''.join([None,None])

TypeError: sequence item 0: expected str instance, NoneType found